In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1261799,2021-05-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1261800,2021-05-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1261801,2021-05-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1261802,2021-05-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
33627,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33629,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33631,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33633,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33635,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
775755,2021-05-10,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804
775757,2021-05-11,Arkansas,Arkansas,5001,2105,36.00,5000,Arkansas,AR,Arkansas,State,3017804
775759,2021-05-12,Arkansas,Arkansas,5001,2109,36.00,5000,Arkansas,AR,Arkansas,State,3017804
775761,2021-05-13,Arkansas,Arkansas,5001,2110,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1261799,2021-05-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1261800,2021-05-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1261801,2021-05-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1261802,2021-05-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
479,2021-05-14,Snohomish,Washington,37721,574.00,53061,WA,County,822083,4588.47,69.82
956,2021-05-14,Cook,Illinois,545785,10669.00,17031,IL,County,5150233,10597.29,207.16
1432,2021-05-14,Orange,California,271134,5028.00,6059,CA,County,3175692,8537.79,158.33
1907,2021-05-14,Maricopa,Arizona,543928,9947.00,4013,AZ,County,4485414,12126.60,221.76
2382,2021-05-14,Los Angeles,California,1237652,24083.00,6037,CA,County,10039107,12328.31,239.89
...,...,...,...,...,...,...,...,...,...,...,...
1261071,2021-05-14,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08
1261285,2021-05-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1261468,2021-05-14,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1261647,2021-05-14,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
479,2021-05-14,Snohomish,Washington,37721,574.00,53061,WA,County,822083,4588.47,69.82,69.82,4588.47
956,2021-05-14,Cook,Illinois,545785,10669.00,17031,IL,County,5150233,10597.29,207.16,207.16,10597.29
1432,2021-05-14,Orange,California,271134,5028.00,6059,CA,County,3175692,8537.79,158.33,158.33,8537.79
1907,2021-05-14,Maricopa,Arizona,543928,9947.00,4013,AZ,County,4485414,12126.60,221.76,221.76,12126.60
2382,2021-05-14,Los Angeles,California,1237652,24083.00,6037,CA,County,10039107,12328.31,239.89,239.89,12328.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261071,2021-05-14,Wheeler,Oregon,32,1.00,41069,OR,County,1332,2402.40,75.08,75.08,2402.40
1261285,2021-05-14,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1261468,2021-05-14,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1261647,2021-05-14,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
811192,2021-05-14,Hale,Alabama,2236,77.00,1065,AL,County,14651,15261.76,525.56,525.56,15261.76,1
883037,2021-05-14,Clarke,Alabama,3513,61.00,1025,AL,County,23622,14871.73,258.23,258.23,14871.73,2
634581,2021-05-14,Lowndes,Alabama,1396,53.00,1085,AL,County,9726,14353.28,544.93,544.93,14353.28,3
472056,2021-05-14,Franklin,Alabama,4262,82.00,1059,AL,County,31362,13589.69,261.46,261.46,13589.69,4
562619,2021-05-14,Etowah,Alabama,13890,353.00,1055,AL,County,102268,13581.96,345.17,345.17,13581.96,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195530,2021-05-14,Platte,Wyoming,630,11.00,56031,WY,County,8393,7506.26,131.06,131.06,7506.26,19
774114,2021-05-14,Converse,Wyoming,1003,17.00,56009,WY,County,13822,7256.55,122.99,122.99,7256.55,20
962599,2021-05-14,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21
1071495,2021-05-14,Niobrara,Wyoming,161,2.00,56027,WY,County,2356,6833.62,84.89,84.89,6833.62,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
634581,2021-05-14,Lowndes,Alabama,1396,53.00,1085,AL,County,9726,14353.28,544.93,544.93,14353.28,3,1
811192,2021-05-14,Hale,Alabama,2236,77.00,1065,AL,County,14651,15261.76,525.56,525.56,15261.76,1,2
269756,2021-05-14,Walker,Alabama,7259,277.00,1127,AL,County,63521,11427.72,436.08,436.08,11427.72,28,3
560955,2021-05-14,Clay,Alabama,1569,57.00,1027,AL,County,13235,11854.93,430.68,430.68,11854.93,20,4
684378,2021-05-14,Greene,Alabama,926,34.00,1063,AL,County,8111,11416.59,419.18,419.18,11416.59,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682721,2021-05-14,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
938586,2021-05-14,Uinta,Wyoming,2239,13.00,56041,WY,County,20226,11069.91,64.27,64.27,11069.91,5,20
962599,2021-05-14,Lincoln,Wyoming,1412,12.00,56023,WY,County,19830,7120.52,60.51,60.51,7120.52,21,21
268490,2021-05-14,Teton,Wyoming,3765,9.00,56039,WY,County,23464,16045.86,38.36,38.36,16045.86,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4588.47,19,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4588.47,19,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4588.47,19,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4588.47,19,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.82,4588.47,19,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258845,2021-05-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1258846,2021-05-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1258847,2021-05-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1258848,2021-05-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
807828,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15261.76,2,1,1.00
807829,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15261.76,2,1,0.00
807830,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15261.76,2,1,0.00
807831,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15261.76,2,1,0.00
807832,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,525.56,15261.76,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978723,2021-05-10,Crook,Wyoming,431,12.00,56011,WY,County,7584,5683.02,158.23,158.23,5696.20,6,23,2.00
978724,2021-05-11,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,1.00
978725,2021-05-12,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,0.00
978726,2021-05-13,Crook,Wyoming,432,12.00,56011,WY,County,7584,5696.20,158.23,158.23,5696.20,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
632039,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14353.28,1,3,1.00,0.00
632040,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14353.28,1,3,0.00,0.00
632041,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14353.28,1,3,0.00,0.00
632042,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14353.28,1,3,0.00,0.00
632043,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14353.28,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629959,2021-05-10,Albany,Wyoming,4365,12.00,56001,WY,County,38880,11226.85,30.86,30.86,11350.31,23,4,36.00,0.00
629960,2021-05-11,Albany,Wyoming,4373,12.00,56001,WY,County,38880,11247.43,30.86,30.86,11350.31,23,4,8.00,0.00
629961,2021-05-12,Albany,Wyoming,4388,12.00,56001,WY,County,38880,11286.01,30.86,30.86,11350.31,23,4,15.00,0.00
629962,2021-05-13,Albany,Wyoming,4406,12.00,56001,WY,County,38880,11332.30,30.86,30.86,11350.31,23,4,18.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
322052,2021-05-14,Imperial,California,28468,728.00,6025,CA,County,181215,15709.52,401.73,401.73,15709.52,1,2,25.00,3.00
2382,2021-05-14,Los Angeles,California,1237652,24083.00,6037,CA,County,10039107,12328.31,239.89,239.89,12328.31,2,5,532.00,16.00
160306,2021-05-14,San Bernardino,California,296903,4722.00,6071,CA,County,2180085,13618.87,216.60,216.60,13618.87,3,4,64.00,7.00
564235,2021-05-14,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
71657,2021-05-14,Stanislaus,California,62188,1061.00,6099,CA,County,550660,11293.36,192.68,192.68,11293.36,5,9,46.00,0.00
33192,2021-05-14,Riverside,California,299793,4594.00,6065,CA,County,2470546,12134.69,185.95,185.95,12134.69,6,6,228.00,0.00
57005,2021-05-14,San Joaquin,California,73310,1394.00,6077,CA,County,762148,9618.87,182.90,182.90,9618.87,7,18,43.00,3.00
88854,2021-05-14,Tulare,California,49830,840.00,6107,CA,County,466195,10688.66,180.18,180.18,10688.66,8,10,-171.00,1.00
32324,2021-05-14,Fresno,California,101862,1687.00,6019,CA,County,999101,10195.37,168.85,168.85,10195.37,9,14,24.00,-1.00
511293,2021-05-14,Merced,California,31919,461.00,6047,CA,County,277680,11494.89,166.02,166.02,11494.89,10,8,5.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1195081,2021-05-14,Lassen,California,5706,24.00,6035,CA,County,30573,18663.53,78.50,78.50,18663.53,34,1,5.00,0.00
322052,2021-05-14,Imperial,California,28468,728.00,6025,CA,County,181215,15709.52,401.73,401.73,15709.52,1,2,25.00,3.00
684320,2021-05-14,Kings,California,23008,246.00,6031,CA,County,152940,15043.81,160.85,160.85,15043.81,11,3,9.00,0.00
160306,2021-05-14,San Bernardino,California,296903,4722.00,6071,CA,County,2180085,13618.87,216.60,216.60,13618.87,3,4,64.00,7.00
2382,2021-05-14,Los Angeles,California,1237652,24083.00,6037,CA,County,10039107,12328.31,239.89,239.89,12328.31,2,5,532.00,16.00
33192,2021-05-14,Riverside,California,299793,4594.00,6065,CA,County,2470546,12134.69,185.95,185.95,12134.69,6,6,228.00,0.00
201573,2021-05-14,Kern,California,109186,1374.00,6029,CA,County,900202,12129.06,152.63,152.63,12129.06,14,7,9.00,0.00
511293,2021-05-14,Merced,California,31919,461.00,6047,CA,County,277680,11494.89,166.02,166.02,11494.89,10,8,5.00,0.00
71657,2021-05-14,Stanislaus,California,62188,1061.00,6099,CA,County,550660,11293.36,192.68,192.68,11293.36,5,9,46.00,0.00
88854,2021-05-14,Tulare,California,49830,840.00,6107,CA,County,466195,10688.66,180.18,180.18,10688.66,8,10,-171.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
322052,2021-05-14,Imperial,California,28468,728.00,6025,CA,County,181215,15709.52,401.73,401.73,15709.52,1,2,25.00,3.00
2382,2021-05-14,Los Angeles,California,1237652,24083.00,6037,CA,County,10039107,12328.31,239.89,239.89,12328.31,2,5,532.00,16.00
160306,2021-05-14,San Bernardino,California,296903,4722.00,6071,CA,County,2180085,13618.87,216.60,216.60,13618.87,3,4,64.00,7.00
564235,2021-05-14,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,0.00,0.00
71657,2021-05-14,Stanislaus,California,62188,1061.00,6099,CA,County,550660,11293.36,192.68,192.68,11293.36,5,9,46.00,0.00
33192,2021-05-14,Riverside,California,299793,4594.00,6065,CA,County,2470546,12134.69,185.95,185.95,12134.69,6,6,228.00,0.00
57005,2021-05-14,San Joaquin,California,73310,1394.00,6077,CA,County,762148,9618.87,182.90,182.90,9618.87,7,18,43.00,3.00
88854,2021-05-14,Tulare,California,49830,840.00,6107,CA,County,466195,10688.66,180.18,180.18,10688.66,8,10,-171.00,1.00
32324,2021-05-14,Fresno,California,101862,1687.00,6019,CA,County,999101,10195.37,168.85,168.85,10195.37,9,14,24.00,-1.00
511293,2021-05-14,Merced,California,31919,461.00,6047,CA,County,277680,11494.89,166.02,166.02,11494.89,10,8,5.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,401.73,15709.52,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4222,05/10/21,Lassen,5698,24.00,18637.36,78.50,78.50,18663.53,34,1,0.00,0.00
4223,05/11/21,Lassen,5701,24.00,18647.17,78.50,78.50,18663.53,34,1,3.00,0.00
4224,05/12/21,Lassen,5701,24.00,18647.17,78.50,78.50,18663.53,34,1,0.00,0.00
4225,05/13/21,Lassen,5701,24.00,18647.17,78.50,78.50,18663.53,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,401.73,15709.52,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,401.73,15709.52,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4308,05/10/21,Merced,31821,459.00,11459.59,165.30,166.02,11494.89,10,8,78.00,1.00
4309,05/11/21,Merced,31859,460.00,11473.28,165.66,166.02,11494.89,10,8,38.00,1.00
4310,05/12/21,Merced,31898,460.00,11487.32,165.66,166.02,11494.89,10,8,39.00,0.00
4311,05/13/21,Merced,31914,461.00,11493.09,166.02,166.02,11494.89,10,8,16.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)